In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
!unzip -uq "/content/drive/My Drive/Colab Notebooks/Boltzmann Machine/Boltzmann_Machines.zip" -d "/content/drive/My Drive/Colab Notebooks/AutoEncoders based recommendation system"

In [0]:
# importing datasets

movies=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Boltzmann Machine/Boltzmann_Machines/ml-1m/movies.dat',sep='::',header=None, engine='python',encoding='latin-1')
users=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Boltzmann Machine/Boltzmann_Machines/ml-1m/users.dat',sep='::',header=None, engine='python',encoding='latin-1')
ratings=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Boltzmann Machine/Boltzmann_Machines/ml-1m/ratings.dat',sep='::',header=None, engine='python',encoding='latin-1')

In [5]:
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [6]:
users

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [7]:
ratings

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [0]:
train_set=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Boltzmann Machine/Boltzmann_Machines/ml-100k/u1.base', delimiter='\t')
train_set=np.array(train_set, dtype='int')
test_set=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Boltzmann Machine/Boltzmann_Machines/ml-100k/u1.test', delimiter='\t')
test_set=np.array(test_set,dtype='int')

In [9]:
train_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [10]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [0]:
nb_users=int(max(max(train_set[:,0]),max(test_set[:,0])))
nb_movies=int(max(max(train_set[:,1]),max(test_set[:,1])))

In [12]:
nb_users

943

In [13]:
nb_movies

1682

In [0]:
def convert(data):
  new_data=[]
  for id_users in range(1,nb_users+1):
    id_movies=data[:,1][data[:,0]==id_users]
    id_ratings=data[:,2][data[:,0]==id_users]
    ratings=np.zeros(nb_movies)
    ratings[id_movies-1]=id_ratings
    new_data.append(list(ratings))
  return new_data

In [0]:
train_set=convert(train_set)

In [0]:
test_set=convert(test_set)

In [17]:
np.shape(train_set)

(943, 1682)

In [18]:
np.shape(test_set)

(943, 1682)

In [0]:
train_set=torch.FloatTensor(train_set)
test_set=torch.FloatTensor(test_set)

In [20]:
train_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [21]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [0]:
# neural network architecture creation

class StackAutoEncoder(nn.Module):
  def __init__(self,):
    super(StackAutoEncoder, self).__init__()
    self.fullconn1=nn.Linear(nb_movies,20)  #start full connection, nb_movies: input neurons, 20: hidden layer and output of nb_movies
    self.fullconn2=nn.Linear(20,10)
    self.fullconn3=nn.Linear(10,20)
    self.fullconn4=nn.Linear(20,nb_movies)  
    self.activation=nn.Sigmoid()

  def forward(self,x):
    x=self.activation(self.fullconn1(x))
    x=self.activation(self.fullconn2(x))
    x=self.activation(self.fullconn3(x))
    x=(self.fullconn4(x))
    return x
sae=StackAutoEncoder()
loss_fun=nn.MSELoss()  #defining the loss function(mean squared error)
optimizer=optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [39]:
# training the model

nb_epoch=250
for i in range (1,nb_epoch+1):
  train_loss=0
  s=0.
  for id_users in range(nb_users):
    input=Variable(train_set[id_users]).unsqueeze(0)  # adding additional dimension at index pos 0 after the train_set
    target=input.clone()  # input and target dims are same
    if torch.sum(target.data>0)>0 : # atleast one movie has been rated
       output=sae(input)
       target.require_grad=False # no optimizers are used at the target end
       output[target==0]=0
       loss=loss_fun(target, output)
       mean_corrector=nb_movies/float(torch.sum(target.data>0)+1e-10)
       loss.backward()  # back propagation
       train_loss+=np.sqrt(loss.data*mean_corrector)
       s+=1.
       optimizer.step()  
    
  print("Epoch: "+str(i)+ " Loss: "+str(train_loss/s))



Epoch: 1 Loss: tensor(1.7656)
Epoch: 2 Loss: tensor(1.0965)
Epoch: 3 Loss: tensor(1.0533)
Epoch: 4 Loss: tensor(1.0383)
Epoch: 5 Loss: tensor(1.0309)
Epoch: 6 Loss: tensor(1.0266)
Epoch: 7 Loss: tensor(1.0239)
Epoch: 8 Loss: tensor(1.0219)
Epoch: 9 Loss: tensor(1.0210)
Epoch: 10 Loss: tensor(1.0195)
Epoch: 11 Loss: tensor(1.0192)
Epoch: 12 Loss: tensor(1.0183)
Epoch: 13 Loss: tensor(1.0178)
Epoch: 14 Loss: tensor(1.0174)
Epoch: 15 Loss: tensor(1.0172)
Epoch: 16 Loss: tensor(1.0169)
Epoch: 17 Loss: tensor(1.0167)
Epoch: 18 Loss: tensor(1.0165)
Epoch: 19 Loss: tensor(1.0164)
Epoch: 20 Loss: tensor(1.0164)
Epoch: 21 Loss: tensor(1.0158)
Epoch: 22 Loss: tensor(1.0160)
Epoch: 23 Loss: tensor(1.0158)
Epoch: 24 Loss: tensor(1.0160)
Epoch: 25 Loss: tensor(1.0155)
Epoch: 26 Loss: tensor(1.0158)
Epoch: 27 Loss: tensor(1.0154)
Epoch: 28 Loss: tensor(1.0153)
Epoch: 29 Loss: tensor(1.0130)
Epoch: 30 Loss: tensor(1.0114)
Epoch: 31 Loss: tensor(1.0103)
Epoch: 32 Loss: tensor(1.0080)
Epoch: 33 Loss: t

In [44]:
test_loss=0 # testing model
s=0.
for id_users in range(nb_users):
    input=Variable(train_set[id_users]).unsqueeze(0)  # adding additional dimension at index pos 0 after the train_set
    target=input.clone()  # input and target dims are same
    if torch.sum(target.data>0)>0 : # atleast one movie has been rated
       output=sae(input)
       target.require_grad=False # no optimizers are used at the target end
       output[target==0]=0
       loss=loss_fun(target, output)
       mean_corrector=nb_movies/float(torch.sum(target.data>0)+1e-10)
       test_loss+=np.sqrt(loss.data*mean_corrector)
       s+=1.
       optimizer.step()  
    
print("Test_Loss: "+str(test_loss/s))

Test_Loss: tensor(0.8896)
